<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 40px; margin-top: 0;">
    <div style="flex: 0 0 auto; margin-left: 0; margin-bottom: 0; margin-top: 0;">
        <img src="./pics/ucsd-logo.png" alt="UCSD Logo" style="width: 179px; margin-bottom: 0px; margin-top: 20px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/ndp-logo.png" alt="NDP Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
    <div style="flex: 0 0 auto; margin-left: auto; margin-bottom: 0; margin-top: 20px;">
        <img src="./pics/hydroframe.png" alt="HydroFrame Logo" style="width: 200px; margin-bottom: 0px;">
    </div>
</div>

<h1 style="text-align: center; font-size: 48px; margin-top: 0;">HydroFrame Demo</h1>

The ParFlow-CONUS1 model is the first generation national ParFlow model. You can learn more about the domain and the model on the HydroFrame website. The conus1_domain dataset contains all of the input fields for the model as well as additional pre and post processing files that can be useful for data analysis. [Click here for more information on the ParFlow-CONUS1 model](https://hydroframe.org/parflow-conus1).

Original resource can be discovered in the National Data Platform's catalog [here](https://nationaldataplatform.org/dataset/inputs-for-baseline-parflow-conus1-simulations).

Official documentation to this dataset with can be found [here](https://hf-hydrodata.readthedocs.io/en/latest/gen_conus1_domain.html)
### Import Libraries

In [ ]:
import hf_hydrodata as hf
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

### Registering a PIN

To access this data, you need to register an account and generate a PIN, go to [this site](https://hydrogen.princeton.edu/). 

In [ ]:
hf.register_api_pin(
    email="your_email@email.edu", # Your email here
    pin="####" # Your PIN here
)

### Helper Functions

In [ ]:
def get_conus1_var(var):
    """Load a static CONUS1 variable as a NumPy array, or return None if pftxt."""
    # Inspect catalog
    meta_opts = {
        "dataset": "conus1_domain",
        "variable": var,
        "temporal_resolution": "static",
    }
    meta = hf.get_catalog_entry(meta_opts)
    file_type = meta.get("file_type")
    grid = meta.get("grid", "conus1")
    print(f"{var}: file_type={file_type}, grid={grid}")

    if file_type == "pftxt":
        print(
            f"'{var}' is stored as ParFlow text (pftxt), "
            "which hf_hydrodata does not yet auto-load as an array. "
            "Skipping."
        )
        return None

    # Load the data (returns NumPy array)
    opts_data = {
        "dataset": "conus1_domain",
        "variable": var,
        "temporal_resolution": "static",
        "grid": grid,
    }
    arr = hf.get_gridded_data(opts_data)
    return arr

def to_2d(arr):
    """If array is 3D (e.g. (nz, ny, nx)), take the first layer for plotting."""
    if arr is None:
        return None
    return arr[0] if arr.ndim == 3 else arr


We will load variables that we have confirmed are available in `pfb` format.

In [ ]:
da = get_conus1_var("drainage_area")
sx = get_conus1_var("slope_x")
sy = get_conus1_var("slope_y")
pme = get_conus1_var("pme")
ssph = get_conus1_var("ss_pressure_head")
pf_ind = get_conus1_var("pf_indicator")

da2d   = to_2d(da)
sx2d   = to_2d(sx)
sy2d   = to_2d(sy)
pme2d  = to_2d(pme)
ssph2d = to_2d(ssph)
pf2d   = to_2d(pf_ind)

print("Shapes:",
      "drainage_area", da2d.shape if da2d is not None else None,
      "slope_x", sx2d.shape if sx2d is not None else None)


### Setting the grid

According to the [documentation](https://hf-hydrodata.readthedocs.io/en/latest/gen_conus1_domain.html), the extent and resolution are the following:

- Grid: conus1
    - Spacial Resolution: 1000 meters
    - XY Grid Spacial Extent: 3342 x 1888
    - LatLon Spacial Exent: -121.47939483437318, 31.651836025255015, -76.09875469594509, 50.49802132270979
    - Origin (meters): -1885055.4995, -604957.0654
    - Projection: +proj=lcc +lat_1=33 +lat_2=45 +lon_0=-96.0 +lat_0=39 +a=6378137.0 +b=6356752.31

In [ ]:
# Grid / projection info
xmin, ymin = -1885055.4995, -604957.0654
dx = dy = 1000.0
nx, ny = 3342, 1888  # x, y

xmax = xmin + nx * dx
ymax = ymin + ny * dy

extent = [xmin, xmax, ymin, ymax]
extent


### Visualization: Drainage Area

In [ ]:
log_da = np.log10(da2d + 1e-6)  # avoid log(0)

vmin = np.nanpercentile(log_da, 5)
vmax = np.nanpercentile(log_da, 95)

plt.figure(figsize=(10, 5))
plt.imshow(
    log_da,
    origin="lower",
    extent=extent,    
    cmap="YlGnBu",
    vmin=vmin,
    vmax=vmax,
)
plt.xlabel("x (m, LCC projection)")
plt.ylabel("y (m, LCC projection)")
plt.title("log10(Drainage Area) – CONUS1 Domain")
plt.colorbar(label="log10(km²)")
plt.gca().set_aspect("equal") 
plt.show()

### Plot: drainage area and slope across the model domain

In [ ]:
# 1. Derived quantities
slope_mag = np.sqrt(sx2d**2 + sy2d**2)
log_da_full = np.log10(da2d + 1e-6)

# 2. Mask to valid cells
mask = (
    np.isfinite(log_da_full) &
    np.isfinite(slope_mag) &
    (da2d > 0)
)

print("Number of valid pixels:", mask.sum())

if mask.sum() == 0:
    print("Mask is empty – check your data arrays (da2d, sx2d, sy2d).")
else:
    log_da = log_da_full[mask]
    slope = slope_mag[mask]

    # Optional: subsample for speed / clarity
    max_points = 200_000
    if log_da.size > max_points:
        idx = np.random.choice(log_da.size, size=max_points, replace=False)
        log_da = log_da[idx]
        slope = slope[idx]

    # 3. Basic EDA stats
    print("Drainage area (log10)  – mean:", float(np.mean(log_da)), "std:", float(np.std(log_da)))
    print("Slope magnitude        – mean:", float(np.mean(slope)),  "std:", float(np.std(slope)))

    corr = np.corrcoef(log_da, slope)[0, 1]
    print("Correlation (log10(DA), slope):", float(corr))

    # 4. Hexbin EDA viz
    plt.figure(figsize=(6, 5))
    hb = plt.hexbin(
        log_da,
        slope,
        gridsize=70,
        mincnt=20, 
        bins="log"
    )
    plt.xlabel("log10(Drainage area + 1e-6)")
    plt.ylabel("Slope magnitude")
    plt.title("Hydrologic Landscape EDA:\nDrainage Area vs Slope (CONUS1)")
    cb = plt.colorbar(hb)
    cb.set_label("log10(pixel count)")
    plt.tight_layout()
    plt.show()


Small drainage areas tend to be steeper (headwaters/ridges), while large drainage areas tend to be flatter (valleys and big river corridors).

## References

- Maxwell, R. M., Condon, L. E., & Kollet, S. J. (2015). *A high-resolution simulation of groundwater and surface water over most of the continental US with the integrated hydrologic model ParFlow v3.* Geoscientific Model Development, 8(3), 923–937. https://doi.org/10.5194/gmd-8-923-2015  
- Condon, L. E., Hering, A. S., & Maxwell, R. M. (2015). *Quantitative assessment of groundwater controls across major US river basins using a multi-model regression algorithm.* Advances in Water Resources, 82, 106–123. https://doi.org/10.1016/j.advwatres.2015.04.008  
- Condon, L. E., & Maxwell, R. M. (2015). *Evaluating the relationship between topography and groundwater using outputs from a continental-scale integrated hydrology model.* Water Resources Research, 51(8), 6602–6621. https://doi.org/10.1002/2014WR016774  
- Maxwell, R. M., Condon, L. E., Kollet, S. J., Maher, K., Haggerty, R., & Forrester, M. M. (2016). *The imprint of climate and geology on the residence times of groundwater.* Geophysical Research Letters, 43(2), 701–708. https://doi.org/10.1002/2015GL066916